In [15]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.pipeline import Pipeline

In [16]:
train_df = pd.read_csv('raw_data/fulltrain.csv', header=None)
train_df.columns = ['label', 'text']
print(train_df.head())
print(train_df.describe())

   label                                               text
0      1  A little less than a decade ago, hockey fans w...
1      1  The writers of the HBO series The Sopranos too...
2      1  Despite claims from the TV news outlet to offe...
3      1  After receiving 'subpar' service and experienc...
4      1  After watching his beloved Seattle Mariners pr...
              label
count  48854.000000
mean       2.487432
std        1.110903
min        1.000000
25%        1.000000
50%        3.000000
75%        3.000000
max        4.000000


In [17]:
# 2-way dataset - satire and hoax
labels = [1, 2]
train_df = train_df[train_df['label'].isin(labels)]

train_df.describe()

,label
count,20989.000000
mean,1.330745
std,0.470492
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,2.000000


In [18]:
X_train, X_test, y_train, y_test = train_test_split(train_df['text'], train_df['label'])

### Random Forest

In [19]:
tfidf_random_forest = TfidfVectorizer(sublinear_tf=True, stop_words='english', tokenizer=word_tokenize)

random_forest_features = tfidf_random_forest.fit_transform(X_train).toarray()
random_forest_features.shape

In [20]:
rf = RandomForestClassifier(n_estimators=50, max_depth=3, random_state=0).fit(random_forest_features, y_train)

# rf_test = tfidf_random_forest.fit_transform(X_test).toarray()

y_pred = rf.predict(random_forest_features)
print(f"Accuracy: {accuracy_score(y_train, y_pred)}")
print(f"F1 score: {f1_score(y_train, y_pred, average='macro')}")

## Linear Support Vector Machine

In [21]:
# CountVectorizer - counts number of times a word appears in text
# TfidfTransformer - weights words from count matrix (via CountVectorizer)
sgd = Pipeline([('vect', CountVectorizer(tokenizer=word_tokenize, stop_words='english')),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=0, max_iter=5, tol=None)),
               ])

sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"F1 score: {f1_score(y_test, y_pred, average='macro')}")

Accuracy: 0.9799923780487805
F1 score: 0.9768968209356461


## balancedtest.csv Results

In [22]:
test_df = pd.read_csv('raw_data/balancedtest.csv', header=None)
test_df.columns = ['label', 'text']

test_df = test_df[test_df['label'].isin(labels)]

X_test = test_df['text']
y_test = test_df['label']

y_pred = sgd.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"F1 score: {f1_score(y_test, y_pred, average='macro')}")

Accuracy: 0.8066666666666666
F1 score: 0.8049183886169207


In [23]:
y_pred = rf.predict(tfidf_random_forest.fit_transform(X_test).toarray())

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"F1 score: {f1_score(y_test, y_pred, average='macro')}")